In [1]:
import pandas
from itertools import product
import sys, argparse, os
import numpy as np
from math import log, ceil
from scipy.stats import multinomial, chi2
from math import factorial
import re
import os
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
folder_list = ['Bergsten_2013', 
               'Broughton_2013',
               'Brown_2012', 
               'Cannon_2016_dna']

In [3]:
training_dataset_prefix = "../training_small"

In [4]:
dic={'A':'1', 'T':'2', 'C':'3', 'G':'4', '-':'0', '?':'0'}

In [5]:
all_data_list = []
all_dists_list_cub = []
all_dists_list_flatten = []

In [6]:
BRANCH_NUM = 100
SEQUENCE_LEN = 1000

OUTPUT_DIST_NUM = int((BRANCH_NUM - 1) * BRANCH_NUM / 2)

In [7]:
all_seq_data = np.zeros((BRANCH_NUM, SEQUENCE_LEN, 1))

In [8]:
# data preprocessing
for folder in folder_list:
    file_path = os.path.join(training_dataset_prefix, folder)
    files_list = os.listdir(file_path)
    for file in files_list:
        if file.find('.nex.treefile.dist') > 0:
            file_base_name = file[0:file.find('.nex.treefile.dist')]
#             print("******************")
#             print(file_base_name)            
            #######################
            # Input Processing !! #
            #######################
            seq_data_raw = open(os.path.join(file_path, file_base_name+'.nex'))
            seq_data = seq_data_raw.readlines()[6:]
            all_seq_data_list = []
            all_seq_data = np.zeros((BRANCH_NUM,SEQUENCE_LEN,1))
            for idx, line in enumerate(seq_data):
                curr_line = line.split()
                if len(curr_line) < 2:
                    break
#                 print('curr_line: ', curr_line)
                seq_data_str = curr_line[1]
#                 print(seq_data_str)
#                 print('===============')
#                 print(seq_data_str)
                for item in dic:
                    seq_data_str = seq_data_str.replace(item, dic[item])
                #replace any remaining letter chars with 0                
                seq_data_str = re.sub(r'[A-Z]', r'0', seq_data_str)
#                 print((seq_data_str))
                seq_data_list = list(seq_data_str)
                if len(seq_data_list) > SEQUENCE_LEN:
                    seq_data_list = seq_data_list[0:SEQUENCE_LEN-1]
                all_seq_data[idx,0:len(seq_data_list),0] = seq_data_list
            all_data_list.append(all_seq_data)
#             print('all_seq_data.shape: ', all_seq_data.shape)

            ########################
            # Output Processing !! #
            ########################
            seq_dists_raw = open(os.path.join(file_path, file_base_name+'.nex.treefile.dist'))
            species_number = int(seq_dists_raw.readline())
            seq_dists = seq_dists_raw.readlines()[0:]            
            all_seq_dists_cub = np.zeros((100,100,1))
            all_seq_dists_flatten = []
            for idx, line in enumerate(seq_dists):                
                curr_line = line.split()
                if len(curr_line) < 2:
                    break

                ### 3 dimension
                dist_values_array_cub = np.array(curr_line[1:])
                dist_values_array_cub = dist_values_array_cub.astype('float')
                all_seq_dists_cub[idx,0:dist_values_array_cub.shape[0],0] = dist_values_array_cub
                ### flatten
                dist_values_array_flatten = np.array(curr_line[1:])
                dist_values_array_flatten = dist_values_array_flatten[idx+1:species_number]
                dist_values_array_flatten = dist_values_array_flatten.astype('float')
                all_seq_dists_flatten = np.concatenate([all_seq_dists_flatten, dist_values_array_flatten])
                
            ### 3 dimension
            all_dists_list_cub.append(all_seq_dists_cub)
            
            ### flatten 
            result = np.zeros((OUTPUT_DIST_NUM))
            result[:all_seq_dists_flatten.shape[0]] = all_seq_dists_flatten
            all_dists_list_flatten.append(result)

In [9]:
training_data = np.stack(all_data_list,axis=0)
training_dists_cub = np.stack(all_dists_list_cub,axis=0)
training_dists_flatten = np.stack(all_dists_list_flatten,axis=0)

print('training_data.shape: ', training_data.shape)
print('training_dists_cub.shape: ', training_dists_cub.shape)
print('training_dists_flatten.shape: ', training_dists_flatten.shape)

training_data.shape:  (500, 100, 1000, 1)
training_dists_cub.shape:  (500, 100, 100, 1)
training_dists_flatten.shape:  (500, 4950)


In [10]:
np.save('./output/training_data.npy', training_data)
np.save('./output/training_dists_cub.npy', training_dists_cub)
np.save('./output/training_dists_flatten.npy', training_dists_flatten)